In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pprint import pprint
from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

# Data

In [3]:
from data_ML import  get_data

In [4]:
N = 5
DIRECTION = 'is_turnpt_upward'
CACHE = True
ADD_DIFF = 0
PCT_TRAIN = 0.75
y, X, idx, num_train = get_data(N, DIRECTION, CACHE, ADD_DIFF, PCT_TRAIN)
num_test = len(y) - num_train

transformed OHLCV at frequency 1D
reading cache of TXF in 1D OHLCV


/home/cyyen/miniconda3/envs/stock_path_tracker/lib/python3.8/site-packages/pandas/core/groupby/groupby.py:980: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis, keys=keys)


# long-only strategy
- time point: 
    - t0 收盤時，得到 s0 訊號
    - t+1 開盤進場（目前簡化先用 t0 收盤價代替）
- predict TP_upward

# Model: Logistics regression as baseline
- reference: https://towardsdatascience.com/weighted-logistic-regression-for-imbalanced-dataset-9a5cd88e68b

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score

In [6]:
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [7]:
X_scaled = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=1-PCT_TRAIN, random_state=0)

## search hyperparameters

## best model

In [8]:
%%time
w = {0: 0.03, 1: 1}
model = LogisticRegression(random_state=13, max_iter=10**3, class_weight=w)
model.fit(X_train, y_train)

CPU times: user 271 ms, sys: 14.9 ms, total: 286 ms
Wall time: 39.8 ms


LogisticRegression(class_weight={0: 0.03, 1: 1}, max_iter=1000, random_state=13)

In [9]:
y_pred = model.predict(X_test)

# performance
print(f'Accuracy Score: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix:')
display(pd.DataFrame(confusion_matrix(y_test, y_pred), index=['False', 'True'], columns=['Negative', 'Positve']))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred)}')

Accuracy Score: 0.5983086680761099
Confusion Matrix:


,Negative,Positve
False,262,189
True,1,21


Area Under Curve: 0.7677383592017739
Recall score: 0.9545454545454546


In [10]:
accurency_train = model.score(X_train, y_train, sample_weight=y_train*100+1)
accurency_test = model.score(X_test, y_test, sample_weight=y_test*100+1)
assert (accurency_test - accurency_train) < 0.1, 'accurency differene between training and testing is larger than 10% '

In [11]:
if __name__ == '__main__':
    from sklearn.model_selection import GridSearchCV

    w = [
        {0: i/1000, 1: 1}
        for i in range(25, 35)
    ]
    crange = np.arange(0.5, 20.0, 0.5)
    hyperparam_grid = {
        "class_weight": w , 
    #     "penalty": ["l1", "l2"],
        "C": crange,
        "fit_intercept": [True, False],
    }
    pprint(hyperparam_grid)

    lg3 = LogisticRegression(random_state=13, max_iter=10**3)
    # define evaluation procedure
    grid = GridSearchCV(lg3,hyperparam_grid, scoring="recall", cv=100, refit=True)
    grid.fit(X_scaled, y)

    print(f'Best score: {grid.best_score_} with param: {grid.best_params_}')

{'C': array([ 0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,
        6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5, 11. ,
       11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. , 16.5,
       17. , 17.5, 18. , 18.5, 19. , 19.5]),
 'class_weight': [{0: 0.025, 1: 1},
                  {0: 0.026, 1: 1},
                  {0: 0.027, 1: 1},
                  {0: 0.028, 1: 1},
                  {0: 0.029, 1: 1},
                  {0: 0.03, 1: 1},
                  {0: 0.031, 1: 1},
                  {0: 0.032, 1: 1},
                  {0: 0.033, 1: 1},
                  {0: 0.034, 1: 1}],
 'fit_intercept': [True, False]}


GridSearchCV(cv=100,
             estimator=LogisticRegression(max_iter=1000, random_state=13),
             param_grid={'C': array([ 0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,
        6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5, 11. ,
       11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. , 16.5,
       17. , 17.5, 18. , 18.5, 19. , 19.5]),
                         'class_weight': [{0: 0.025, 1: 1}, {0: 0.026, 1: 1},
                                          {0: 0.027, 1: 1}, {0: 0.028, 1: 1},
                                          {0: 0.029, 1: 1}, {0: 0.03, 1: 1},
                                          {0: 0.031, 1: 1}, {0: 0.032, 1: 1},
                                          {0: 0.033, 1: 1}, {0: 0.034, 1: 1}],
                         'fit_intercept': [True, False]},
             scoring='recall')

Best score: 0.96 with param: {'C': 0.5, 'class_weight': {0: 0.025, 1: 1}, 'fit_intercept': True}
